In [1]:
# Import libraries--------------------------------------------------------------------------------------

import os 
import numpy as np 
import pandas as pd 
import napari
import re 

from scipy import ndimage as nd
from skimage.measure import label, regionprops 
from skimage.segmentation import expand_labels
from skimage.morphology import binary_erosion, square, opening
from tifffile import imread, imwrite
from tqdm import tqdm

In [2]:
# Define functions---------------------------------------------------------------------------------------

def cyto_expand(img, px): 
    cyto_shell = expand_labels(img, distance = px) 
    cyto_buffer = expand_labels(img, distance = 2) 
    cyto = np.subtract(cyto_shell, cyto_buffer) 

    return cyto

def roi_quant(img, img_gast_label, img_edt, img_405, img_488, img_568, img_647):
    df = pd.DataFrame() 
    
    roi_id = [] 
    area = []
    gast_id = [] 
    dist_mean = [] 
    dist_max = [] 
    centroid_y = []
    centroid_x = []
    intensity_405 = [] 
    intensity_488 = [] 
    intensity_568 = [] 
    intensity_647 = [] 
    
    gast_props = regionprops(img, intensity_image = img_gast_label)
    edt_props = regionprops(img, intensity_image = img_edt)
    img405_props = regionprops(img, intensity_image = img_405)
    img488_props = regionprops(img, intensity_image = img_488)
    img568_props = regionprops(img, intensity_image = img_568)
    img647_props = regionprops(img, intensity_image = img_647)
    
    for roi in tqdm(range(len(gast_props))):
        roi_id.append(gast_props[roi].label)
        area.append(gast_props[roi].area)
        gast_id.append(gast_props[roi].max_intensity)
        dist_mean.append(edt_props[roi].mean_intensity)
        dist_max.append(edt_props[roi].max_intensity)
        centroid_y.append(gast_props[roi].centroid[0])
        centroid_x.append(gast_props[roi].centroid[1])
        intensity_405.append(img405_props[roi].mean_intensity)
        intensity_488.append(img488_props[roi].mean_intensity)
        intensity_568.append(img568_props[roi].mean_intensity)
        intensity_647.append(img647_props[roi].mean_intensity)
        
    df['roi_id'] = roi_id
    df['area'] = area
    df['gast_id'] = gast_id
    df['dist_mean'] = dist_mean
    df['dist_max'] = dist_max
    df['centroid_y'] = centroid_y
    df['centroid_x'] = centroid_x
    df['intensity_405'] = intensity_405
    df['intensity_488'] = intensity_488
    df['intensity_568'] = intensity_568
    df['intensity_647'] = intensity_647
    
    return df

In [3]:
# Import path and image list----------------------------------------------------------------------------

base_path = 'K:/Analysis/[NCI] [LGI] Sergio Ruiz/Bechara Saykali/images/gastruloids/2023/07_14'

tmp_list = os.listdir(base_path)
img_list = [] 
for tmp in tmp_list:
    if re.search('_405.tif', tmp):
        img_list.append(tmp.replace('_405.tif', ''))
        
print(img_list)
print(len(img_list))

['230714_H9DHB_Gastr_BMP4_CDX2_Ncad_001 - Stitched', '230714_H9DHB_Gastr_BMP4_CDX2_Ncad_002 - Stitched', '230714_H9DHB_Gastr_BMP4_CDX2_Ncad_003 - Stitched', '230714_H9DHB_Gastr_BMP4_CDX2_Ncad_004 - Stitched', '230714_H9DHB_Gastr_BMP4_CDX2_Ncad_005 - Stitched', '230714_H9DHB_Gastr_BMP4_CDX2_Ncad_006 - Stitched', '230714_H9DHB_Gastr_BMP4_CDX2_Ncad_007 - Stitched', '230714_H9DHB_Gastr_BMP4_CDX2_Ncad_008 - Stitched', '230714_H9DHB_Gastr_BMP4_CDX2_Ncad_009 - Stitched', '230714_H9DHB_Gastr_BMP4_SOX2_TFAP2c_001 - Stitched', '230714_H9DHB_Gastr_BMP4_SOX2_TFAP2c_002 - Stitched', '230714_H9DHB_Gastr_BMP4_SOX2_TFAP2c_003 - Stitched', '230714_H9DHB_Gastr_BMP4_SOX2_TFAP2c_004 - Stitched', '230714_H9DHB_Gastr_BMP4_SOX2_TFAP2c_005 - Stitched', '230714_H9DHB_Gastr_BMP4_SOX2_TFAP2c_006 - Stitched', '230714_H9DHB_Gastr_BMP4_SOX2_TFAP2c_007 - Stitched', '230714_H9DHB_Gastr_BMP4_SOX2_TFAP2c_008 - Stitched', '230714_H9DHB_Gastr_BMP4_SOX2_TFAP2c_009 - Stitched', '230714_H9DHB_Gastr_noBMP4_CDX2_Ncad_011 - St

In [4]:
# Main function--------------------------------------------------------------------------------------------

for img_name in img_list:
    print(img_name)
    
    img_nuc_path = os.path.join(base_path, img_name + '_568_cp_masks.tif')
    img_nuc = imread(img_nuc_path)
    
    img_gast_path = os.path.join(base_path, img_name + '_568_gast_mask.tif')
    img_gast = imread(img_gast_path)
    
    img_edt_path = os.path.join(base_path, img_name + '_568_edt_map.tif')
    img_edt = imread(img_edt_path)
    
    img_405_path = os.path.join(base_path, img_name + '_405.tif')
    img_405 = imread(img_405_path)
    
    img_488_path = os.path.join(base_path, img_name + '_488.tif')
    img_488 = imread(img_488_path)
    
    img_568_path = os.path.join(base_path, img_name + '_568.tif')
    img_568 = imread(img_568_path)
    
    img_647_path = os.path.join(base_path, img_name + '_647.tif')
    img_647 = imread(img_647_path)
    
    print('Generating gastruloid label')
    img_gast_label = label(img_gast)
    img_gast_label_path = os.path.join(base_path, img_name + '_568_gast_label.tif')
    imwrite(img_gast_label_path, img_gast_label)
    
    print('Generating cytoplasmic regions')
    img_cyto = cyto_expand(img_nuc, 12)
    img_cyto_path = os.path.join(base_path, img_name + '_568_cyto_label.tif')
    imwrite(img_cyto_path, img_cyto)
    
    print('Quantifying gastruloids')
    df_gast = roi_quant(img_gast_label, img_gast_label, img_edt, img_405, img_488, img_568, img_647)
    df_gast_path = os.path.join(base_path, img_name + '_gastruloid.csv')
    df_gast.to_csv(df_gast_path, header = True)
    
    print('Quantifying nuclei')
    df_nuc = roi_quant(img_nuc, img_gast_label, img_edt, img_405, img_488, img_568, img_647)
    df_nuc_path = os.path.join(base_path, img_name + '_nuclei.csv')
    df_nuc.to_csv(df_nuc_path, header = True)
    
    print('Quantifying cytoplasm')
    df_cyto = roi_quant(img_cyto, img_gast_label, img_edt, img_405, img_488, img_568, img_647)
    df_cyto_path = os.path.join(base_path, img_name + '_cyto.csv')
    df_cyto.to_csv(df_cyto_path, header = True)
    
print('Quantification complete')    

230714_H9DHB_Gastr_BMP4_CDX2_Ncad_001 - Stitched
Generating gastruloid label
Generating cytoplasmic regions
Quantifying gastruloids


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.16s/it]


Quantifying nuclei


100%|████████████████████████████████████████████████████████████████████████████| 4295/4295 [00:01<00:00, 2900.96it/s]


Quantifying cytoplasm


...

Quantifying nuclei


100%|████████████████████████████████████████████████████████████████████████████| 5118/5118 [00:01<00:00, 2961.00it/s]


Quantifying cytoplasm


100%|████████████████████████████████████████████████████████████████████████████| 5118/5118 [00:01<00:00, 2695.93it/s]


Quantification complete
